In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
%matplotlib inline
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights, resnet34, resnext50_32x4d
from torch.optim.lr_scheduler import StepLR

# used source from: https://pytorch.org/hub/pytorch_vision_resnext/
#cuda sematic:https://pytorch.org/docs/stable/notes/cuda.html

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    
])

In [10]:
#for both DBI&SSD data path in drive
# !unzip -q -o "/content/drive/MyDrive/DBIsubset.zip" -d "/content/"
# !unzip -o "/content/drive/MyDrive/SSDsubset.zip" -d "/content/"
!unzip -q -j "/content/drive/MyDrive/DBIsubset.zip" -d "/content/DBIsubset/"
!unzip -q -j "/content/drive/MyDrive/SSDsubset.zip" -d "/content/SSDsubset/"


!find /content/ -name "__MACOSX" -exec rm -rf {} +
!find /content/ -name "._*" -exec rm -rf {} +
!find /content/ -name ".DS_Store" -exec rm -rf {} +
!find /content/ -name ".*" -exec rm -rf {} +
!ls /content/DBIsubset

path1 = '/content/DBIsubset'
path2 = '/content/SSDsubset'
DBIDATA = ImageFolder(path1, transform=transform)
SSDDATA = ImageFolder(path2, transform=transform)


replace /content/DBIsubset/chihuahua14.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace /content/SSDsubset/n02099601_1028.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
rm: cannot remove '/content/drive/.shortcut-targets-by-id': Operation canceled
rm: cannot remove '/content/drive/.file-revisions-by-id': Operation canceled
rm: cannot remove '/content/drive/.Trash-0': Directory not empty
rm: cannot remove '/content/drive/.Encrypted/MyDrive': Operation canceled
rm: cannot remove '/content/drive/.Encrypted/.shortcut-targets-by-id': Operation canceled
rm: cannot remove '/content/drive/.Encrypted/.shortcut-targets-by-id': Operation canceled
bernese100.jpg	      chihuahua3.jpg   collie78.jpg	 husky32.jpg	  labrador6.jpg
bernese10.jpg	      chihuahua40.jpg  collie7.jpg	 husky33.jpg	  labrador70.jpg
bernese11.jpg	      chihuahua41.jpg  collie80.jpg	 husky34.jpg	  labrador71.jpg
bernese12.jpg	      chihuahua42.jpg  collie81.jpg	 husky35.jpg	  labrador72.jpg
bernese13.jpg	      chihuahua44.jpg

In [24]:
batch = 32
dbi_train_size = int(0.7 * len(DBIDATA))
dbi_val_size = int(0.15 * len(DBIDATA))
dbi_test_size = len(DBIDATA) - dbi_train_size - dbi_val_size
print("DBI folder:", DBIDATA.classes)
print("SDD folder:", SSDDATA.classes)

train_size = int(0.7 * len(DBIDATA))
val_size = int(0.15 * len(DBIDATA))
test_size = len(DBIDATA) - dbi_train_size - dbi_val_size

#DBI
dbi_train, dbi_val, dbi_test = random_split(DBIDATA, [dbi_train_size, dbi_val_size, dbi_test_size])

dbi_train_loader = DataLoader(dbi_train, batch_size = batch, shuffle=True)
dbi_val_loader = DataLoader(dbi_val, batch_size = batch, shuffle=False)
dbi_test_loader = DataLoader(dbi_test, batch_size=batch, shuffle=False)

#SSD
ssd_loader = DataLoader(SSDDATA, batch_size=batch, shuffle=False)

print("SSDLOADER:", ssd_loader)
pre_trained_models = {
    "ResNet18": resnet18(weights='IMAGENET1K_V1'),
    "ResNet34": resnet34(weights='IMAGENET1K_V1'),
    "ResNeXt32": resnext50_32x4d(weights='IMAGENET1K_V1')
}

DBI folder: ['bernese_mountain_dog', 'border_collie', 'chihuahua', 'golden_retriever', 'labrador_retriever', 'pug', 'siberian_husky']
SDD folder: ['n02085620-Chihuahua', 'n02099601-golden_retriever', 'n02099712-Labrador_retriever', 'n02106166-Border_collie', 'n02107683-Bernese_mountain_dog', 'n02110185-Siberian_husky', 'n02110958-pug']
SSDLOADER: <torch.utils.data.dataloader.DataLoader object at 0x78e3876137d0>


In [26]:
from logging import lastResort
def fineTune(model, train_loader, val_loader, ssd_loader, total_class):

  rate = 0.0001

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr= rate)
  scheduler = StepLR(optimizer, step_size = 5, gamma = 0.1)

  train_acc, validate_acc = [], []
  for epoch in range(10):
      model.train()
      total, valid_trainm, correct = 0, 0,0

      for img, labels in train_loader:
        img, labels = img.to(device), labels.to(device)
        optimizer.zero_grad()

        to_return = model(img)
        loss = criterion(to_return, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(to_return.data, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

      scheduler.step()

      train_accuracy = 100 * correct /total
      train_acc.append(train_accuracy)


      #for validation now
      model.eval()
      total_v, valid_v= 0, 0
      with torch.no_grad():
        for img_v, labels_v in val_loader:
            img_v, val_labels = img_v.to(device), labels_v.to(device)
            val_outputs = model(img_v)
            _, preds_v = torch.max(val_outputs.data, 1)
            total_v += val_labels.size(0)
            valid_v += (preds_v == labels_v).sum().item()

      epoch_val_acc = 100 * valid_v / total_v
      validate_acc.append(epoch_val_acc)
      print(f'Epoch [{epoch+1}/10], Train-Acc: {train_accuracy:.2f}%, Validation-Acc: {epoch_val_acc:.2f}%')

  def eval(loader):
    model.eval()
    correct, total = 0,0
    with torch.no_grad():
      for img, labels in loader:
        img, labels = img.to(device), labels.to(device)
        to_return = model(img)
        _, preds = torch.max(to_return.data, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()
    return 100 * correct / total

  dbi_test_acc = eval(dbi_test_loader)
  sdd_acc = eval(ssd_loader)

  print(f"DBI Test Accuracy: {dbi_test_acc:.2f}%")
  print(f"SDD Accuracy: {sdd_acc:.2f}%")

  return dbi_test_acc, sdd_acc

num_classes = len(DBIDATA.classes)
results = {}

for model_name, model in pre_trained_models.items():
  print(f"\nFine-tuning {model_name}...")

# Replace final fully connected layer to match DBI classes
  model.fc = nn.Linear(model.fc.in_features, num_classes)

  dbi_acc, sdd_acc = fineTune(model, dbi_train_loader, dbi_val_loader, ssd_loader, num_classes)
  results[model_name] = {"DBI Test Accuracy": dbi_acc, "SDD Accuracy": sdd_acc}

print("\nCross-Performance Analysis:")
for model_name, metrics in results.items():
  print(f"{model_name}: DBI Test Accuracy = {metrics['DBI Test Accuracy']:.2f}%, SDD Accuracy = {metrics['SDD Accuracy']:.2f}%")




Fine-tuning ResNet18...
Epoch [1/10], Train-Acc: 71.68%, Validation-Acc: 92.71%
Epoch [2/10], Train-Acc: 99.56%, Validation-Acc: 94.79%
Epoch [3/10], Train-Acc: 100.00%, Validation-Acc: 93.75%
Epoch [4/10], Train-Acc: 100.00%, Validation-Acc: 96.88%
Epoch [5/10], Train-Acc: 99.34%, Validation-Acc: 94.79%
Epoch [6/10], Train-Acc: 100.00%, Validation-Acc: 95.83%
Epoch [7/10], Train-Acc: 100.00%, Validation-Acc: 95.83%
Epoch [8/10], Train-Acc: 100.00%, Validation-Acc: 93.75%
Epoch [9/10], Train-Acc: 100.00%, Validation-Acc: 95.83%
Epoch [10/10], Train-Acc: 100.00%, Validation-Acc: 94.79%
DBI Test Accuracy: 93.88%
SDD Accuracy: 1.14%

Fine-tuning ResNet34...
Epoch [1/10], Train-Acc: 71.90%, Validation-Acc: 95.83%
Epoch [2/10], Train-Acc: 99.34%, Validation-Acc: 96.88%
Epoch [3/10], Train-Acc: 98.89%, Validation-Acc: 95.83%
Epoch [4/10], Train-Acc: 99.56%, Validation-Acc: 95.83%
Epoch [5/10], Train-Acc: 100.00%, Validation-Acc: 94.79%
Epoch [6/10], Train-Acc: 100.00%, Validation-Acc: 95.83